# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

from texttable import Texttable
import latextable

In [3]:
import sys
sys.path.insert(1, '../sar_dirichlet')
import dirichlet_regression

In [4]:
from func_test import cos_similarity, rmse_aitchison

In [5]:
scipy.__version__

'1.10.1'

# Loading Dataset

In [6]:
arctic = pd.read_csv('Data Dirichlet/ArcticLake.csv')

In [7]:
Y_arctic = arctic.iloc[:,:3]
X_arctic = arctic.iloc[:,3]

In [8]:
Y_arctic = np.array(Y_arctic)

In [9]:
X_arctic = np.array([[j] for j in X_arctic])

In [10]:
Z_arctic = np.ones(len(X_arctic)).reshape((39,1))

In [11]:
n_features = 1
n_classes = 3

In [12]:
X = np.array(X_arctic)
Y = np.array(Y_arctic)

X = StandardScaler().fit(X).transform(X)

In [13]:
Z = Z_arctic
gamma_0 = [0.]

In [14]:
n,K = X.shape
J = Y.shape[1]

# Order 1

In [15]:
distance_matrix = scipy.spatial.distance_matrix(X_arctic,X_arctic)
W_arctic_cont = np.zeros(np.shape(distance_matrix))
W_arctic_cont[distance_matrix < 28] = 1
# replace the 1 on the diagonal by 0
np.fill_diagonal(W_arctic_cont,0)
# scaling the matrix, so that the sum of each row is 1
W_arctic_cont = W_arctic_cont/W_arctic_cont.sum(axis=1)[:,None]

In [16]:
neighbors = NearestNeighbors(n_neighbors=29).fit(X)
W_arctic_dist = neighbors.kneighbors_graph(X,mode='distance').toarray()
W_arctic_dist[W_arctic_dist>0] = 1/W_arctic_dist[W_arctic_dist>0]
W_arctic_dist = W_arctic_dist/W_arctic_dist.sum(axis=1)[:,None]

# Inference tests

In [17]:
%%time
dirichRegressor = dirichlet_regression.dirichletRegressor()
dirichRegressor.fit(X, Y, parametrization='alternative', gamma_0 = gamma_0, Z=Z)

Optimization terminated successfully.
Wall time: 11.1 ms


In [18]:
dirichlet_regression.dirichlet_loglikelihood(dirichRegressor.mu,dirichRegressor.phi,Y)

77.72809940035866

In [19]:
dirichRegressor.beta

array([[1.044688  , 0.42575255],
       [1.08824776, 1.55837907]])

In [20]:
dirichRegressor.compute_hessian(X,Y,Z)

In [21]:
np.diag(dirichRegressor.hess)

array([-148.43568465, -115.49153564, -147.58486218, -128.28546153,
        -45.43222872])

In [22]:
np.sqrt(np.diag(np.linalg.pinv(-dirichRegressor.hess)))

array([0.13184403, 0.14043332, 0.15368295, 0.16312287, 0.16226657])

In [23]:
dirichRegressor.gamma[0]

2.626560915133771

In [24]:
# standard errors
np.sqrt(np.diag(cov))

NameError: name 'cov' is not defined

In [116]:
# z-values : \hat{\beta} / se
print(f"z-values for beta : {dirichRegressor.beta.flatten()/np.sqrt(np.diag(cov))[:-1]}")
print(f"z-values for gamma : {dirichRegressor.gamma[0] / np.sqrt(np.diag(cov))[-1]}")

z-values for beta : [-3.86622119 -9.03654411  7.08112563  9.55338604]
z-values for gamma : 16.186695954424245


## Spatial contiguity order 1

In [64]:
%%time
dirichRegressor_sc_1 = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor_sc_1.fit(X, Y, W=W_arctic_cont, parametrization='alternative', gamma_0 = gamma_0, Z=Z)

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Wall time: 54.2 ms


In [71]:
dirichRegressor_sc_1.inference(X, Y, Z, W_arctic_cont, display=True)

-----
Estimated parameter beta_11 = 2.12, se = 0.14, CI 95% = [1.85 ; 2.38],  p-value = 0.0
-----
Estimated parameter beta_12 = 0.86, se = 0.14, CI 95% = [0.6 ; 1.13],  p-value = 0.0
-----
Estimated parameter beta_21 = 1.99, se = 0.15, CI 95% = [1.69 ; 2.29],  p-value = 0.0
-----
Estimated parameter beta_22 = 2.85, se = 0.17, CI 95% = [2.52 ; 3.18],  p-value = 0.0
-----
Estimated parameter gamma_1 = 2.76, se = 0.16, CI 95% = [2.45 ; 3.08],  p-value = 0.0
-----
Estimated parameter rho = -1.0, se = 0.2, CI 95% = [-1.39 ; -0.61],  p-value = 0.0


## Spatial contiguity order 2

In [25]:
X = np.array(X_arctic)
X = StandardScaler().fit(X).transform(X)

In [26]:
X_2 = np.ones((39,2))
X_2[:,0] = X[:,0]
X_2[:,1] = X[:,0]**2

In [66]:
%%time
dirichRegressor_sc_2 = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor_sc_2.fit(X_2, Y, W=W_arctic_cont, parametrization='alternative', gamma_0 = gamma_0, Z=Z)

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
Wall time: 92.6 ms


In [70]:
dirichRegressor_sc_2.inference(X_2, Y, Z, W_arctic_cont, display=True)

-----
Estimated parameter beta_11 = 2.53, se = 0.16, CI 95% = [2.22 ; 2.85],  p-value = 0.0
-----
Estimated parameter beta_12 = 1.57, se = 0.16, CI 95% = [1.25 ; 1.89],  p-value = 0.0
-----
Estimated parameter beta_21 = 1.78, se = 0.11, CI 95% = [1.56 ; 2.0],  p-value = 0.0
-----
Estimated parameter beta_22 = 2.97, se = 0.15, CI 95% = [2.68 ; 3.27],  p-value = 0.0
-----
Estimated parameter beta_31 = -0.49, se = 0.11, CI 95% = [-0.71 ; -0.27],  p-value = 0.0
-----
Estimated parameter beta_32 = -0.88, se = 0.13, CI 95% = [-1.14 ; -0.63],  p-value = 0.0
-----
Estimated parameter gamma_1 = 3.01, se = 0.16, CI 95% = [2.7 ; 3.32],  p-value = 0.0
-----
Estimated parameter rho = -1.0, se = 0.17, CI 95% = [-1.34 ; -0.66],  p-value = 0.0


In [58]:
p-valeur, beta chapeau, se, ecart type, intervalle confiance
spatial contiguity order 1 & 2
gamma et rho

SyntaxError: invalid syntax (<ipython-input-58-f71afd3cc2d7>, line 1)

## Spatial distance

In [59]:
%%time
dirichRegressor_sd = dirichlet_regression.dirichletRegressor(spatial=True)
dirichRegressor_sd.fit(X, Y, W=W_arctic_dist, parametrization='alternative', gamma_0 = gamma_0, Z=Z)

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Wall time: 53.9 ms


In [60]:
dirichlet_regression.dirichlet_loglikelihood(dirichRegressor_sd.mu,dirichRegressor_sd.phi,Y)

77.72859550588747

In [61]:
dirichRegressor_sd.compute_hessian(X,Y,Z,W_arctic_dist)

In [62]:
np.diag(dirichRegressor_sd.hess)

array([-148.46121327, -115.5266222 , -147.58466579, -128.29355169,
        -45.43246597, -109.90484265])

In [63]:
dirichRegressor_sd.beta

array([[1.06451567, 0.4328314 ],
       [1.10685771, 1.58427968]])

In [64]:
dirichRegressor_sd.rho

-0.01988173139158971